In [ ]:
#import library
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA

# Define paths
train_csv_path = 'Human Action Recognition/Training_set.csv'
train_images_path = 'Human Action Recognition/train'

# Load the training data
train_data = pd.read_csv(train_csv_path)

# Function to load and vectorize images
def load_and_vectorize_images(image_folder, dataframe):
    images = []
    labels = []
    for _, row in dataframe.iterrows():
        image_path = os.path.join(image_folder, row['filename'])
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Reading in grayscale
        if image is not None:
            image = cv2.resize(image, (64, 64))  # Resize images
            images.append(image.flatten())  # Flatten the image
            labels.append(row['label'])
    return np.array(images), np.array(labels)

# Load and vectorize images
X, y = load_and_vectorize_images(train_images_path, train_data)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% of the variance
X_pca = pca.fit_transform(X_scaled)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_pca, y_encoded, test_size=0.2, random_state=42)

# Create and train the SVM model
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

# Predict on the validation set
y_pred = svm_model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred, target_names=label_encoder.classes_)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


Accuracy: 0.22023809523809523
Classification Report:
                    precision    recall  f1-score   support

           calling       0.21      0.12      0.15       173
          clapping       0.21      0.10      0.13       160
           cycling       0.19      0.28      0.23       186
           dancing       0.29      0.33      0.31       180
          drinking       0.19      0.18      0.19       152
            eating       0.20      0.31      0.24       151
          fighting       0.23      0.37      0.29       186
           hugging       0.16      0.17      0.17       151
          laughing       0.33      0.17      0.22       179
listening_to_music       0.18      0.14      0.16       176
           running       0.23      0.27      0.25       155
           sitting       0.23      0.21      0.22       163
          sleeping       0.29      0.33      0.31       162
           texting       0.16      0.09      0.11       183
      using_laptop       0.22      0.23      0

In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from skimage import exposure
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Define paths
train_csv_path = 'Human Action Recognition/Training_set.csv'
train_images_path = 'Human Action Recognition/train'

# Load the training data
train_data = pd.read_csv(train_csv_path)

# Define a common size for image resizing
common_image_size = (64, 64)

# Function to load, resize, and vectorize images
def load_resize_and_vectorize_images(image_folder, dataframe):
    images = []
    labels = []
    for _, row in dataframe.iterrows():
        image_path = os.path.join(image_folder, row['filename'])
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Reading in grayscale
        if image is not None:
            image = cv2.resize(image, common_image_size)  # Resize images to a common size
            images.append(image.flatten())  # Flatten the image
            labels.append(row['label'])
    return np.array(images), np.array(labels)

# Load, resize, and vectorize images
X, y = load_resize_and_vectorize_images(train_images_path, train_data)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Create and train the SVM model
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

# Predict on the validation set
y_pred = svm_model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred, target_names=label_encoder.classes_)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


Accuracy: 0.2253968253968254
Classification Report:
                    precision    recall  f1-score   support

           calling       0.20      0.10      0.14       173
          clapping       0.21      0.11      0.14       160
           cycling       0.20      0.30      0.24       186
           dancing       0.30      0.33      0.31       180
          drinking       0.19      0.18      0.18       152
            eating       0.21      0.36      0.27       151
          fighting       0.24      0.38      0.29       186
           hugging       0.16      0.17      0.16       151
          laughing       0.33      0.17      0.23       179
listening_to_music       0.18      0.14      0.16       176
           running       0.24      0.31      0.27       155
           sitting       0.21      0.20      0.20       163
          sleeping       0.29      0.32      0.30       162
           texting       0.17      0.10      0.12       183
      using_laptop       0.22      0.22      0.

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Define paths
train_csv_path = 'Human Action Recognition/Training_set.csv'
train_images_path = 'Human Action Recognition/train'

# Load the training data
train_data = pd.read_csv(train_csv_path)

# Define fixed feature lengths
fixed_sift_length = 128  # Choose based on your dataset and experiments
fixed_hog_length = 3780  # Typical length for HOG features; adjust as needed

# Function to compute and standardize SIFT features
def compute_sift_features(image):
    sift = cv2.SIFT_create()
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    keypoints, sift_features = sift.detectAndCompute(gray_image, None)
    if sift_features is not None:
        sift_features = sift_features.flatten()
        if len(sift_features) > fixed_sift_length:
            sift_features = sift_features[:fixed_sift_length]  # Truncate
        elif len(sift_features) < fixed_sift_length:
            sift_features = np.pad(sift_features, (0, fixed_sift_length - len(sift_features)), 'constant')  # Pad
    else:
        sift_features = np.zeros(fixed_sift_length)
    return sift_features

# Function to compute and standardize HOG features
def compute_hog_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog_features = hog(gray_image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=False, multichannel=False)
    if len(hog_features) > fixed_hog_length:
        hog_features = hog_features[:fixed_hog_length]  # Truncate
    elif len(hog_features) < fixed_hog_length:
        hog_features = np.pad(hog_features, (0, fixed_hog_length - len(hog_features)), 'constant')  # Pad
    return hog_features

# Extract features
features_list = []
labels = []

for _, row in train_data.iterrows():
    image_path = os.path.join(train_images_path, row['filename'])
    image = cv2.imread(image_path)
    if image is not None:
        sift_features = compute_sift_features(image)
        hog_features = compute_hog_features(image)
        combined_features = np.hstack((sift_features, hog_features))
        features_list.append(combined_features)
        labels.append(row['label'])

# Convert the lists to NumPy arrays
X = np.array(features_list)
y = np.array(labels)

# Encode labels and normalize features
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Train the SVM model
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = svm_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred, target_names=label_encoder.classes_)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


/var/folders/4w/qh2mw0kn6853yfpwbvg7fhtm0000gn/T/ipykernel_1787/1022023688.py:40: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  hog_features = hog(gray_image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=False, multichannel=False)


Accuracy: 0.2706349206349206
Classification Report:
                    precision    recall  f1-score   support

           calling       0.21      0.21      0.21       173
          clapping       0.21      0.26      0.23       160
           cycling       0.26      0.45      0.33       186
           dancing       0.37      0.26      0.31       180
          drinking       0.16      0.14      0.15       152
            eating       0.31      0.38      0.34       151
          fighting       0.36      0.31      0.34       186
           hugging       0.18      0.13      0.15       151
          laughing       0.28      0.37      0.32       179
listening_to_music       0.19      0.10      0.13       176
           running       0.29      0.26      0.28       155
           sitting       0.23      0.24      0.24       163
          sleeping       0.41      0.35      0.37       162
           texting       0.23      0.11      0.15       183
      using_laptop       0.32      0.45      0.

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Define paths
train_csv_path = 'Human Action Recognition/Training_set.csv'
train_images_path = 'Human Action Recognition/train'

# Load the training data
train_data = pd.read_csv(train_csv_path)

# Define fixed feature lengths
fixed_sift_length = 128  # Choose based on your dataset and experiments
fixed_hog_length = 3780  # Typical length for HOG features; adjust as needed

# Function to compute and standardize SIFT features
def compute_sift_features(image):
    sift = cv2.SIFT_create()
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    keypoints, sift_features = sift.detectAndCompute(gray_image, None)
    if sift_features is not None:
        sift_features = sift_features.flatten()
        if len(sift_features) > fixed_sift_length:
            sift_features = sift_features[:fixed_sift_length]  # Truncate
        elif len(sift_features) < fixed_sift_length:
            sift_features = np.pad(sift_features, (0, fixed_sift_length - len(sift_features)), 'constant')  # Pad
    else:
        sift_features = np.zeros(fixed_sift_length)
    return sift_features

# Function to compute and standardize HOG features
def compute_hog_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog_features = hog(gray_image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=False, multichannel=False)
    if len(hog_features) > fixed_hog_length:
        hog_features = hog_features[:fixed_hog_length]  # Truncate
    elif len(hog_features) < fixed_hog_length:
        hog_features = np.pad(hog_features, (0, fixed_hog_length - len(hog_features)), 'constant')  # Pad
    return hog_features

# Extract features
features_list = []
labels = []

for _, row in train_data.iterrows():
    image_path = os.path.join(train_images_path, row['filename'])
    image = cv2.imread(image_path)
    if image is not None:
        sift_features = compute_sift_features(image)
        hog_features = compute_hog_features(image)
        combined_features = np.hstack((sift_features, hog_features))
        features_list.append(combined_features)
        labels.append(row['label'])

# Convert the lists to NumPy arrays
X = np.array(features_list)
y = np.array(labels)

# Encode labels and normalize features
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA for dimensionality reduction
pca = PCA(n_components=0.95)  # Retain 95% of variance
X_pca = pca.fit_transform(X_scaled)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_pca, y_encoded, test_size=0.2, random_state=42)

# Train the SVM model
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = svm_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred, target_names=label_encoder.classes_)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

/var/folders/4w/qh2mw0kn6853yfpwbvg7fhtm0000gn/T/ipykernel_1787/1618815314.py:40: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  hog_features = hog(gray_image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=False, multichannel=False)


Accuracy: 0.26904761904761904
Classification Report:
                    precision    recall  f1-score   support

           calling       0.20      0.20      0.20       173
          clapping       0.20      0.25      0.22       160
           cycling       0.26      0.45      0.33       186
           dancing       0.35      0.26      0.30       180
          drinking       0.15      0.14      0.15       152
            eating       0.30      0.38      0.34       151
          fighting       0.36      0.31      0.33       186
           hugging       0.19      0.13      0.16       151
          laughing       0.29      0.39      0.33       179
listening_to_music       0.19      0.11      0.14       176
           running       0.29      0.26      0.28       155
           sitting       0.23      0.24      0.24       163
          sleeping       0.38      0.33      0.35       162
           texting       0.27      0.12      0.17       183
      using_laptop       0.32      0.44      0

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog, local_binary_pattern
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Define paths
train_csv_path = 'Human Action Recognition/Training_set.csv'
train_images_path = 'Human Action Recognition/train'

# Load the training data
train_data = pd.read_csv(train_csv_path)

# Define fixed feature lengths
fixed_sift_length = 128
fixed_hog_length = 3780
fixed_lbp_length = 256

# Function to compute and standardize SIFT features
def compute_sift_features(image):
    sift = cv2.SIFT_create()
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    keypoints, sift_features = sift.detectAndCompute(gray_image, None)
    if sift_features is not None:
        sift_features = sift_features.flatten()
    else:
        sift_features = np.zeros(fixed_sift_length)
    # Padding or truncating to fixed length
    return np.pad(sift_features, (0, max(0, fixed_sift_length - len(sift_features))), 'constant')[:fixed_sift_length]

# Function to compute and standardize HOG features
def compute_hog_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog_features = hog(gray_image, orientations=8, pixels_per_cell=(16, 16),
                       cells_per_block=(1, 1), visualize=False, multichannel=False)
    # Padding or truncating to fixed length
    return np.pad(hog_features, (0, max(0, fixed_hog_length - len(hog_features))), 'constant')[:fixed_hog_length]

# Function to compute and standardize LBP features
def compute_lbp_features(image, P=8, R=1):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray_image, P, R, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, P * (P - 1) + 3), range=(0, P * (P - 1) + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    # Padding or truncating to fixed length
    return np.pad(hist, (0, max(0, fixed_lbp_length - len(hist))), 'constant')[:fixed_lbp_length]

# Extract features
features_list = []
labels = []

for _, row in train_data.iterrows():
    image_path = os.path.join(train_images_path, row['filename'])
    image = cv2.imread(image_path)
    if image is not None:
        sift_features = compute_sift_features(image)
        hog_features = compute_hog_features(image)
        lbp_features = compute_lbp_features(image)
        combined_features = np.hstack((sift_features, hog_features, lbp_features))
        features_list.append(combined_features)
        labels.append(row['label'])

# Convert to NumPy arrays and encode labels
X = np.array(features_list)
y = np.array(labels)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA for dimensionality reduction
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X_pca, y_encoded, test_size=0.2, random_state=42)

# SVM model training
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_model.fit(X_train, y_train)

# Prediction and evaluation
y_pred = svm_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred, target_names=label_encoder.classes_)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

/var/folders/4w/qh2mw0kn6853yfpwbvg7fhtm0000gn/T/ipykernel_1787/570420736.py:39: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  hog_features = hog(gray_image, orientations=8, pixels_per_cell=(16, 16),


Accuracy: 0.28293650793650793
Classification Report:
                    precision    recall  f1-score   support

           calling       0.20      0.20      0.20       173
          clapping       0.23      0.28      0.25       160
           cycling       0.31      0.44      0.36       186
           dancing       0.39      0.29      0.34       180
          drinking       0.17      0.14      0.15       152
            eating       0.31      0.39      0.35       151
          fighting       0.38      0.35      0.37       186
           hugging       0.19      0.17      0.18       151
          laughing       0.30      0.39      0.34       179
listening_to_music       0.18      0.11      0.13       176
           running       0.29      0.28      0.28       155
           sitting       0.23      0.25      0.24       163
          sleeping       0.39      0.35      0.37       162
           texting       0.27      0.14      0.18       183
      using_laptop       0.32      0.46      0

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog, local_binary_pattern
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Define paths
train_csv_path = 'Human Action Recognition/Training_set.csv'
train_images_path = 'Human Action Recognition/train'

# Load the training data
train_data = pd.read_csv(train_csv_path)

# Define fixed feature lengths
fixed_sift_length = 128
fixed_hog_length = 3780
fixed_lbp_length = 256

# Function to compute and standardize SIFT features
def compute_sift_features(image):
    sift = cv2.SIFT_create()
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    keypoints, sift_features = sift.detectAndCompute(gray_image, None)
    if sift_features is not None:
        sift_features = sift_features.flatten()
    else:
        sift_features = np.zeros(fixed_sift_length)
    # Padding or truncating to fixed length
    return np.pad(sift_features, (0, max(0, fixed_sift_length - len(sift_features))), 'constant')[:fixed_sift_length]

# Function to compute and standardize HOG features
def compute_hog_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog_features = hog(gray_image, orientations=8, pixels_per_cell=(16, 16),
                       cells_per_block=(1, 1), visualize=False, multichannel=False)
    # Padding or truncating to fixed length
    return np.pad(hog_features, (0, max(0, fixed_hog_length - len(hog_features))), 'constant')[:fixed_hog_length]

# Function to compute and standardize LBP features
def compute_lbp_features(image, P=8, R=1):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray_image, P, R, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, P * (P - 1) + 3), range=(0, P * (P - 1) + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    # Padding or truncating to fixed length
    return np.pad(hist, (0, max(0, fixed_lbp_length - len(hist))), 'constant')[:fixed_lbp_length]

# Extract features
features_list = []
labels = []

for _, row in train_data.iterrows():
    image_path = os.path.join(train_images_path, row['filename'])
    image = cv2.imread(image_path)
    if image is not None:
        sift_features = compute_sift_features(image)
        hog_features = compute_hog_features(image)
        lbp_features = compute_lbp_features(image)
        combined_features = np.hstack((sift_features, hog_features, lbp_features))
        features_list.append(combined_features)
        labels.append(row['label'])

# Convert to NumPy arrays and encode labels
X = np.array(features_list)
y = np.array(labels)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# SVM model training
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_model.fit(X_train, y_train)

# Prediction and evaluation
y_pred = svm_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred, target_names=label_encoder.classes_)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

/var/folders/4w/qh2mw0kn6853yfpwbvg7fhtm0000gn/T/ipykernel_1787/3783595474.py:39: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  hog_features = hog(gray_image, orientations=8, pixels_per_cell=(16, 16),


Accuracy: 0.2841269841269841
Classification Report:
                    precision    recall  f1-score   support

           calling       0.21      0.22      0.22       173
          clapping       0.23      0.29      0.26       160
           cycling       0.30      0.43      0.36       186
           dancing       0.39      0.29      0.34       180
          drinking       0.17      0.14      0.16       152
            eating       0.31      0.40      0.35       151
          fighting       0.38      0.34      0.36       186
           hugging       0.18      0.16      0.17       151
          laughing       0.30      0.40      0.34       179
listening_to_music       0.17      0.09      0.12       176
           running       0.31      0.28      0.30       155
           sitting       0.24      0.25      0.25       163
          sleeping       0.39      0.35      0.37       162
           texting       0.27      0.14      0.18       183
      using_laptop       0.32      0.45      0.

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from PIL import Image
from keras.applications import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from keras.utils import to_categorical

# Define paths
data_folder = "Human Action Recognition/"
train_csv_path = "Human Action Recognition/Training_set.csv"
train_images_path = "Human Action Recognition/train"

# Load the training data
train_csv = pd.read_csv(train_csv_path)

# Initialize VGG16 model for feature extraction
base_model = VGG16(weights='imagenet', include_top=False)

def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(160, 160))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    features = model.predict(img_array)
    return features.flatten()

# Extract features from images
features_list = []
labels_list = []

for _, row in train_csv.iterrows():
    img_path = os.path.join(train_images_path, row['filename'])
    if os.path.exists(img_path):
        features = extract_features(img_path, base_model)
        features_list.append(features)
        labels_list.append(row['label'])

# Convert lists to NumPy arrays
X = np.array(features_list)
y = np.array(labels_list)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train a Kernel SVM
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = svm_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred, target_names=label_encoder.classes_)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


1/1 [==============================] - 0s 83ms/step
Accuracy: 0.6444444444444445
Classification Report:
                    precision    recall  f1-score   support

           calling       0.47      0.41      0.44       173
          clapping       0.65      0.59      0.62       160
           cycling       0.98      0.92      0.95       186
           dancing       0.69      0.74      0.72       180
          drinking       0.59      0.60      0.60       152
            eating       0.75      0.85      0.80       151
          fighting       0.70      0.67      0.69       186
           hugging       0.55      0.68      0.60       151
          laughing       0.62      0.65      0.64       179
listening_to_music       0.50      0.40      0.44       176
           running       0.74      0.71      0.73       155
           sitting       0.52      0.56      0.54       163
          sleeping       0.76      0.77      0.77       162
           texting       0.50      0.42      0.45      